Pytorch implementation of "Attention is all you need!!" paper by Vaswani et al in Pytorch

My recreation of http://nlp.seas.harvard.edu/annotated-transformer/

- Encoder maps an input sequence of symbol representations  (x1,...,xn)  to a sequence of continuous representations  z=(z1,...,zn). 

- Given  z , the decoder then generates an output sequence  (y1,...,ym)  of symbols one element at a time. 

- At each step the model is auto-regressive, consuming the previously generated symbols as additional input when generating the next.

In [8]:
import os
from os.path import exists
import torch
import torch.nn as nn
from torch.nn.functional import log_softmax, pad
import math
import copy
import time
from torch.optim.lr_scheduler import LambdaLR
import pandas as pd
import altair as alt
# from torchtext.data.functional import to_map_style_dataset
from torch.utils.data import DataLoader
from torchtext.vocab import build_vocab_from_iterator
import torchtext.datasets as datasets
import spacy
import GPUtil
import warnings
from torch.utils.data.distributed import DistributedSampler
import torch.distributed as dist
import torch.multiprocessing as mp
from torch.nn.parallel import DistributedDataParallel as DDP


OSError: dlopen(/Users/vivekr/Documents/vivek/projects/.venv/lib/python3.9/site-packages/torchtext/lib/libtorchtext.so, 0x0006): Symbol not found: __ZN3c105ErrorC1ENSt3__112basic_stringIcNS1_11char_traitsIcEENS1_9allocatorIcEEEES7_PKv
  Referenced from: <DF3ABA20-1A43-316C-A7EC-B234DC9E3163> /Users/vivekr/Documents/vivek/projects/.venv/lib/python3.9/site-packages/torchtext/lib/libtorchtext.so
  Expected in:     <69A84A04-EB16-3227-9FED-383D2FE98E93> /Users/vivekr/Documents/vivek/projects/.venv/lib/python3.9/site-packages/torch/lib/libc10.dylib

Overall Encoder Decoder architecture

In [2]:
class EncoderDecoder(nn.Module):

    def __init__(self, encoder, decoder, in_embed, out_embed, generator):
        super().__init__()
        self.encoder = encoder
        self.decoder = decoder
        self.in_embed = in_embed
        self.out_embed = out_embed
        self.generator = generator

    def forward(self, src, tgt, mask, src_mask, tgt_mask):
        '''
            Process masked source and target sequences
        '''
        return self.decode(self.encode(src, src_mask), src_mask, tgt, tgt_mask)
    
    def encode(self, src, src_mask):
        return self.encoder(self.in_embed(src), src_mask)

    def decode(self, memory, src_mask, tgt, tgt_mask):
        return self.decoder(self.out_embed(tgt), memory, src_mask, tgt_mask)



In [3]:
class Generator(nn.Module):
    "Define standard linear + softmax generation step."

    def __init__(self, d_model, vocab):
        super(Generator, self).__init__()
        self.proj = nn.Linear(d_model, vocab)

    def forward(self, x):
        return log_softmax(self.proj(x), dim=-1)